**Lucas-Kanade optical flow**

In [2]:
import cv2 as cv
import numpy as np

#Shi-Tomasi corner detection
feature_params = dict(maxCorners = 300, qualityLevel = 0.2, minDistance = 2, blockSize = 7)

#Lucas-Kanade optical flow process parameters 
lk_params = dict(winSize = (15,15), maxLevel = 2, criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

# Input video
cap = cv.VideoCapture("video1.mp4")

#color to draw flow track
color = (0, 255, 0)

#reading the frames of video
ret, first_frame = cap.read()

# Converts frame to grayscale because we only need the luminance channel for detecting edges 
prev_gray = cv.cvtColor(first_frame, cv.COLOR_BGR2GRAY)

#points good points/pixel
prev = cv.goodFeaturesToTrack(prev_gray, mask = None, **feature_params)
mask = np.zeros_like(first_frame)

while(cap.isOpened()):
    
    ret, frame = cap.read()
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    
    #flow process
    next, status, error = cv.calcOpticalFlowPyrLK(prev_gray, gray, prev, None, **lk_params)
    good_old = prev[status == 1]#position variables
    good_new = next[status == 1]#position variables
    #marking flow tracks
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv.line(mask, (a, b), (c, d), color, 1)
        frame = cv.circle(frame, (a, b), 2, color, -1)
    output = cv.add(frame, mask)
    prev_gray = gray.copy()
    prev = good_new.reshape(-1, 1, 2)
    
    #output function
    cv.imshow("sparse optical flow", output)
    
    #function to destroy window
    if cv.waitKey(10) & 0xFF == ord('q'):
        break

#clears memory
cap.release()
cv.destroyAllWindows()


<ipython-input-2-2460cc6712e4>:39: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  mask = cv.line(mask, (a, b), (c, d), color, 1)
<ipython-input-2-2460cc6712e4>:40: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  frame = cv.circle(frame, (a, b), 2, color, -1)


**Dense Optical Flow**

In [4]:
import cv2 as cv
import numpy as np

#input video 
cap = cv.VideoCapture("video.mp4")

#reading of each frame of video
ret, first_frame = cap.read()
prev_gray = cv.cvtColor(first_frame, cv.COLOR_BGR2GRAY)
mask = np.zeros_like(first_frame)

# Sets image saturation to maximum
mask[..., 1] = 255

while(cap.isOpened()):
    ret, frame = cap.read()
    
    #window shows orignal video frames
    cv.imshow("input", frame)
    
    #converts each frame to grayscale
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    
    #dense optical flow 
    flow = cv.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    magnitude, angle = cv.cartToPolar(flow[..., 0], flow[..., 1])
    mask[..., 0] = angle * 180 / np.pi / 2
    
    # Sets image value according to the optical flow magnitude 
    mask[..., 2] = cv.normalize(magnitude, None, 0, 255, cv.NORM_MINMAX)
    
    # Converts HSV to RGB (BGR) color representation
    rgb = cv.cvtColor(mask, cv.COLOR_HSV2BGR)
    
    #window shows output conversion flow
    cv.imshow("dense optical flow", rgb)
    prev_gray = gray    
    #window distruction function pess 'q'
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

#clears memory
cap.release()
cv.destroyAllWindows()